# BAF complex structure inference
![Baf structure](BAF_struct.jpg)

* 250A = ARID1A
* 250B = (ARID1B)
* 60A = SMARCD1
* 60B = SMARCD2
* 60C = SMARCD3
* BCL7A = BCL7A
* BCL7B = BCL7B
* BCL7C = -BCL7C
* 155 = SMARCC1 
* 170 = SMARCC2
* 57 = SMARCE1 
* BRG1 = SMARCA4 
* BRM = SMARCA2
* 53A = ACTL6A
* $\beta$-actin = (ACTB)
* SS18 = (SS18)
* 47 = SMARCB1
* 45D = DPF2
* (45B) = DPF1
* (45C) = DPF3
* (SS18L1) = SS18L1

* BRD9 = (BRD9)

In [ ]:
using CSV, DataFrames, StatsBase, Plotly, LightGraphs, GraphIO, Distributions

In [ ]:
global const ALPHA = 0.05

In [ ]:
colnames = ["Units"    
 "ACTB"     
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCA4.4"
 "SMARCA4.6"
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]
aridData = CSV.read("ARID1A-data.csv"; delim='\t', header=colnames, datarow=2)

In [ ]:
foreach(x -> aridData[x] = log2.(aridData[x]), names(aridData[:,2:end]))

In [ ]:
aridPval = CSV.read("ARID1A-pval.csv"; delim='\t', header=colnames, datarow=2)
aridPval[1] = aridData[1]
aridPval

We now remove variations where the fold change is not significantly greater than zero.

In [ ]:
for i in 2:length(aridData)
    for j in 1:length(aridData[i])
        # Some values were stored as factors instead of floats, and could not be compared to ALPHA
        try
            if aridPval[j,i] > ALPHA
                aridData[j,i] = 0
            end
        catch e
            if isa(e, MethodError) # In case of type error when comparing the variable to ALPHA 
                if float(string(aridPval[j,i])) > ALPHA # Try converting the faulty variable
                    aridData[j,i] = 0
                end
            end
        end
    end
end

## BAF complex structure
Pulling down ARID1A only capture the BAF complex

In [ ]:
# Join SMARCA4.4 and SMARCA4.6
delete!(aridData, Symbol("SMARCA4.6"))
rename!(aridData, Symbol("SMARCA4.4") => :SMARCA4)

In [ ]:
init_notebook(true)

traceArid = heatmap(
    x=aridData[1],
    y=names(aridData[2:end]),
    z=convert(Array, aridData[:,2:end])
)

#===== Color mapping
We want a linear scale from blue to white (minimal value to zero)
then from white to red (zero to maximal value).
Plotly expect linear scales with endpoints in zero (minimal value)
to 1 (maximal value), therefore we transform the coordinate c in
our scale to plotly's scale p by the following transformation:
p = (c - minVal)/(maxVal - minVal)
=====#
coordZero = -minimum(convert(Array, aridData[:,2:end])) /
    (maximum(convert(Array, aridData[:,2:end])) - minimum(convert(Array, aridData[:,2:end])))
styleArid = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [coordZero, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutArid = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")

plot(traceArid, layoutArid, style=styleArid)

## Create interaction graph

In [ ]:
studyBAFko = convert(Array{String,1}, names(aridData[2:end]))
studyBAFpd = convert(Array{String,1}, aridData[1])
unitDict = Dict(s => i for (i,s) in enumerate(sort(union(studyBAFko, studyBAFpd))))

In [ ]:
effectGraph = SimpleDiGraph()
add_vertices!(effectGraph, length(unitDict))

In [ ]:
# Store the sign of the log2-fold-change associated with each link
edgeTypes = Dict{Tuple, String}() 

# Parse each column
for x = names(aridData[:,2:end])
    for y = 1:length(aridData[x])
        if aridData[y,x] < 0
            add_edge!(effectGraph, unitDict[String(x)], unitDict[String(aridData[y,:Units])])
            edgeTypes[(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "inhibits"
        elseif aridData[y,x] > 0
            add_edge!(effectGraph, unitDict[String(x)], unitDict[String(aridData[y,:Units])])
            edgeTypes[(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "enhances"
        end
    end
end

In [ ]:
"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` given in a dictionary to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and edge labels
'elabs' given in two dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs::Dict{Tuple,String})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tlabel \"", elabs[(s,t)], '"')
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and node class
'nclass' given in two dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs::Dict{Int64,Int64})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t\tclass ", elabs[i])
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

## Genetic algorithm approach
### Subset pull-down graph to known BAF units
### Create a structure graph
### From structure graph to pull-down graph
### Compare pull-down graph
### Mutate a structure graph
### Combine as genetic algorithm

In [ ]:
hugoBAFunits = CSV.read("BAF_genefamily.tsv"; delim='\t')

In [ ]:
# Are all the pulled-down proteins known sub-units of the BAF complex?
all(unit -> unit in hugoBAFunits[2], aridData[1])

In [ ]:
# Which elements should we include in our structural model?
studyBAFunits = [k for k in keys(unitDict) if k in hugoBAFunits[2]]
# How many subunits are we considering?
const M = length(studyBAFunits)

## Define graph Julia struct
Pulldown graphs contain the directed graph of activation/inhibition, the node and edges annotations.  
Structure graphs contain the structural graph, the node annotations and the competition classes of each node.

In [ ]:
mutable struct pulldownGraph
    graph::SimpleDiGraph
    nodes::Dict{Int64,String}
    edges::Dict{Tuple, String}
end

In [ ]:
mutable struct structureGraph
    graph::SimpleGraph
    nodes::Dict{Int64,String}
    competition::Dict{Int64,Int64}
end

## Define constants used by the algorithm

In [ ]:
const inhibitEdge = "inhibits"
const enhanceEdge = "enhances"
# ARID1A should not be deleted
@assert !("ARID1A" in studyBAFko)
    #=====
    When we delete a node from a lightgraph, the node to
    remove is swapped with the last node in the node list.
    To ensure that the index of ARID1A is stable, we make
    sur that it is never knocked out nor the last node.
    =====#
# Remember ARID1A index
const aridIndex = [i for i in 1:length(studyBAFunits) if studyBAFunits[i] == "ARID1A"][1]
# ARID1A should not be the last subunit in the list
@assert aridIndex != length(studyBAFunits)

In [ ]:
[e for e in unitDict if !(e[1] in studyBAFunits)]

In [ ]:
# Link indices to unsorted list of BAF units
unitDictStudy = Dict(enumerate(studyBAFunits))
# Convert node indices from experimental graph to simulated graphs
convertUnitIndex = Dict(unitDict[v] => u for (u,v) in unitDictStudy)
observedEdges = Dict((convertUnitIndex[u[1]], convertUnitIndex[u[2]]) => v for 
        (u,v) in edgeTypes if u[1] in keys(convertUnitIndex) && u[2] in keys(convertUnitIndex))

In [ ]:
studyBAFpdIndices = [ipd for (ipd, pd) in enumerate(studyBAFunits) if pd in studyBAFpd]
studyBAFkoIndices = [iko for (iko, ko) in enumerate(studyBAFunits) if ko in studyBAFko]

## Define graph functions

In [ ]:
"""
Compute pulldown graph corresponding to a
structure graph given as argument
"""
function structureToPulldown(sGraph::structureGraph)
    # The structure graph must include all BAF subunits
    # @assert nv(sGraph.graph) == length(studyBAFunits)
    
    # Initialise a pulldownGraph
    # with the studied nodes and no edges
    pGraph = pulldownGraph(
        SimpleDiGraph(M),
        sGraph.nodes,
        Dict{Tuple, String}()
    )
    
    # Create dict from competitions between units                
    competitionDict = getCompetitionDict(sGraph.competition)
                    
    # For each unit knocked-out
    for iko = studyBAFkoIndices
        # Compute what units are still connected to ARID1A
        pulledComponent = getPulledComponent(sGraph.graph, iko)
        
        # Check what would be observed for each pulled down subunit
        for ipd = studyBAFpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraph, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledComponent)
                        add_pulldown_edge!(inhibitEdge, pGraph, iko, ipd)
                        continue # Look at next pulldowned subunit
                    end
                    # The PD subunit is connected
                    if enhanceIfDisconnectedCompetition!(pGraph, pulledComponent,
                        competitionDict, ipd, iko)
                        # The subunit is enriched
                        continue # Look at next pulldowned subunit
                    end
                elseif !(ipd in pulledComponent)
                    # If a subunit is not in the component connected
                    # to ARID1A, the KO will decrease the quantity of
                    # this subunit that will be pulled-down
                    add_pulldown_edge!(inhibitEdge, pGraph, iko, ipd)
                    continue # Look at next pulldowned subunit
                else
                    # The PD subunit is connected
                    enhanceIfDisconnectedCompetition!(pGraph, pulledComponent,
                        competitionDict, ipd, iko)
                    continue # Look at next pulldowned subunit
                end
            end
        end        
    end
    
    return(pGraph)
end

"""
Add a link to a pulldownGraph
"""
function add_pulldown_edge!(edgeType::String, pGraph::pulldownGraph, from::Int64, to = from)
    add_edge!(pGraph.graph, from, to)
    pGraph.edges[(from, to)] = edgeType
end
                        
"""
Create a dictionary associating a subunit with its competitors
"""
function getCompetitionDict(competition::Dict{Int64,Int64})
    competitionDF = DataFrame(Int64, M, 2)
    for i in 1:M
        competitionDF[i,1] = i
        competitionDF[i,2] = competition[i]
    end
    names!(competitionDF, [:Key, :Value])
    
    competitionDict = Dict{Int64, Array}()
    for df in groupby(competitionDF, :Value)
        for value in df[:Key]
            competitionDict[value] = [i for i in df[:Key] if i != value]
        end
    end
    
    return(competitionDict)
end

"""
Predict enrichment if a KO disconnect a competitor
of a subunit
"""
function enhanceIfDisconnectedCompetition!(pGraph::pulldownGraph, 
        pulledComponent::Array{Int64,1}, competitionDict::Dict{Int64, Array},
        ipd::Int64, iko::Int64)
    # For the KOed subunit
    if ipd in competitionDict[iko]
        add_pulldown_edge!(enhanceEdge, pGraph, iko, ipd)
        return(true) # An edge has been added
    end    
    # For all non-KOed subunit
    for inc = (j for j in 1:(M-1) if !(j in pulledComponent))
        if inc == iko
            # If the subunit has the index 'iko' it is
            # actually the last subunit, that has been
            # swapped with the KOed subunit
            inc = M
        end
        if ipd in competitionDict[inc]
            add_pulldown_edge!(enhanceEdge, pGraph, iko, ipd)
            return(true) # An edge has been added
        end
    end
    return(false) # No edge has been added
end

"""
Return a list of all subunits still connected
to ARID1A after a given KO is performed
"""        
function getPulledComponent(graph::LightGraphs.SimpleGraphs.SimpleGraph{Int64}, iko::Int64)
    perturbGraph = copy(graph)
    rem_vertex!(perturbGraph, iko)
    pulledComponent = Array{Int64,1}
    for component in connected_components(perturbGraph) if aridIndex in component
        return(component)
    end end
end
                        
"""
Enforce the connectivity of a structureGraph
"""
function connectGraph!(sGraph::structureGraph)
    while !is_connected(sGraph.graph)
        mutateAddEdge!(sGraph)
end end
                        
"""
Attribute random competition classes for subunits not
yet present in competition dictionary of a structureGraph
"""
function randomCompetitionGraph!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    for i = 1:M
        # Get the index of all subunit not in the competition dict
        u = map(reverse, unitDictStudy)[studyBAFunits[i]]
        if !(u in keys(competition))
            # Assign random competition class
            competition[u] = rand(1:M)
        end
    end
end

## Define mutation functions

In [ ]:
"""
Mutate a single structure graph
The keywords contain the mutation parameters:
    p_add: add edge probability
    p_del: del edge probability
    p_swp: swap edge probability
    p_cmp: competition class probability
"""
function mutateStructureGraph!(sGraph::structureGraph; 
        p_add = 0.1, p_del = p_add, p_swp = p_add, p_cmp = p_add)
    # Store exit codes of individual mutation functions
    status = 0
    
    # Determine which mutations to perform
    doMutate = rand(4) .< [p_add, p_del, p_swp, p_cmp]
    
    if doMutate[1]
        status += mutateAddEdge!(sGraph)
    end

    if doMutate[2]
        status += mutateDelEdge!(sGraph.graph)
    end

    if doMutate[3]
        status += mutateSwapEdges!(sGraph)
    end

    if doMutate[4]
        status += mutateCompetitors!(sGraph)
    end

    return(status)
end
  
"""
Add an edge to a structure graph
"""
function mutateAddEdge!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    N = nv(graph)
    
    if ne(graph) >= N*(N-1)/2
        # The graph is already complete
        return(1)
    else
        while true
            (a,b) = ceil.(N*rand(2))
            if (a != b) && (add_edge!(graph, a, b))
                # Do not allow self loop
                # Do not allow links between competitors
                if (competition[a] == competition[b])
                    rem_edge!(graph, Int64(a), Int64(b))
                    return(1)
                end
                # Exit if edge sucessfully added
                return(0)
            end
        end
    end
end

"""
Remove an edge to a structure graph
"""
function mutateDelEdge!(graph::LightGraphs.SimpleGraphs.SimpleGraph)
    edgesList = [e for e in edges(graph)]
    edgesIndicesOrder = randperm(length(edgesList))
    for edgeIndex in edgesIndicesOrder
        edgeToRemove = edgesList[edgeIndex]
        rem_edge!(graph, edgeToRemove)
        if is_connected(graph)
            return(0)
        else
            # So structure graph should be kept connected
            # Therefore we put back in the removed edge
            add_edge!(graph, edgeToRemove)
        end
    end
    
    # No edge can be removed without diconnecting the graph
    return(1)
end

"""
Swap edges in a structure graph
"""
function mutateSwapEdges!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    edgesList = [e for e in edges(graph)]
    edgesIndicesOrder = randperm(length(edgesList))
    
    for (indexIndex, edgeIndex) = enumerate(edgesIndicesOrder)
        edge1 = edgesList[edgeIndex]
        edge2 = edgesList[edgesIndicesOrder[1+(indexIndex % length(edgesList))]]
        # Ensure that no self link will be created
        if Tuple(edge1)[1] != Tuple(edge2)[2] && Tuple(edge2)[1] != Tuple(edge1)[2]
            # Start by deleting the old edges
            rem_edge!(graph, edge1)
            rem_edge!(graph, edge2)
            # Then add the new ones if not linking competitors
            if competition[Tuple(edge1)[1]] != competition[Tuple(edge2)[2]]
                add_edge!(graph, Tuple(edge1)[1], Tuple(edge2)[2])
            end
            if competition[Tuple(edge2)[2]] != competition[Tuple(edge1)[2]]
                add_edge!(graph, Tuple(edge2)[1], Tuple(edge1)[2])
            end
            if is_connected(graph)
                return(0)
            else
                # So structure graph should be kept connected
                # Therefore we put back in the removed edges
                add_edge!(graph, edge1)
                add_edge!(graph, edge2)
                # NB: extra edges will stay if any
            end
        end
    end
    
    # No edges can be swapped without diconnecting the graph
    return(1)
end

"""
Mutate competing nodes
"""
function mutateCompetitors!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    # Select node to change competition class
    nodeComp = rand(1:nv(graph))
    # Select new competition class
    newComp = rand(1:nv(graph))
    for n = neighbors(graph, nodeComp)
        if competition[n] == newComp
            # Changing the competition class would lead to linked competitors
            return(1)
        end
    end
    competition[nodeComp] = newComp
    
    return(0)
end

"""
Cross-over between two structure graphs
"""
function crossOverGraphs!(sGraph1::structureGraph, sGraph2::structureGraph)
    return(1)
end

## Genetic algorithm module

In [ ]:
"""
Compute loss for a given structure
compared to observation
"""
function observedLoss(sGraph::structureGraph,
    details::Bool = false)
    pGraph = structureToPulldown(sGraph)
    
    intersectEdges = intersect(pGraph.edges, observedEdges)
    unionEdges = union(pGraph.edges, observedEdges)
    
    if details
        # Return array with Jaccard index
        # length of union and length of  
        return([length(intersectEdges) / length(unionEdges), length(intersectEdges), length(pGraph.edges)])
    else
        # Return Jaccard index
        return([length(intersectEdges) / length(unionEdges)])
    end
end

"""
Generate in place the new generation of 
structure graphs based on their fitness.
Return the fitness array.
"""
function reproduceGeneration!(pop::Array{structureGraph,1},
    details::Bool = false)
    jaccard = map(x -> observedLoss(x,details), pop)
    fitness = map(x -> x[1], jaccard)
    fitness ./= sum(fitness)
    
    sumFitness = sum(fitness) 
    if sumFitness != 1
        fitness[end] += 1 - sumFitness
    end
    # Ensure the cumulative fitnesses is a probability distribution
    
    offspringPerGraph = rand(Multinomial(length(pop), fitness), 1)
    offspring = Array{structureGraph,1}(length(pop))
    
    offspringToFill = 1 # Which is the next index to be filled?
    for (ipop, noff) = enumerate(offspringPerGraph)
        for ioff = 1:noff
            offspring[offspringToFill] = deepcopy(pop[ipop])
            offspringToFill += 1
        end
    end
    
    # Ensure the best structure graph is kept
    bestGraphIndex = findmax(fitness)[2]
    if offspringPerGraph[bestGraphIndex] == 0
        # No offspring for the best graph
        # So we force one
        offspring[1] = deepcopy(pop[bestGraphIndex])
    end
    
    pop .= offspring
        
    return(jaccard)
end

"""
Generate the new generation of structure networks
"""
function newGeneration!(pop::Array{structureGraph,1},
        details::Bool = false;
        p_add = 0.1, p_del = p_add, p_swp = p_add, p_cmp = p_add, p_crs = p_add/10)
    # Fitness-based reproduction
    fitness = reproduceGeneration!(pop, details)
    
    # Mutate potentially each structure network
    map(x -> mutateStructureGraph!(x;
            p_add = p_add, p_del = p_del, p_swp = p_swp, p_cmp = p_cmp), pop)
    
    # Cross-over
#     if rand() < p_crs
#         sGraph1 = rand(pop)
#         sGraph2 = rand(pop)
#         if sGraph1 != sGraph2
#             crossOverGraphs!(sGraph1, sGraph2)
#         end
#     end
    
    return(fitness)
end

## Load results

In [ ]:
using JLD, HDF5
maxFit = 0
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/littIni/500x10000/"
bestPop = Dict{String,Any}()
for runFile = [i for i in readdir(folder) if contains(i, ".jld")]
    popl = load(folder*runFile)
    maxFitPopl = maximum(map(x -> observedLoss(x, false)[1], popl["pop"]))
    if maxFitPopl > maxFit
        bestPop = popl
        maxFit = maxFitPopl
    end
end
quantileFitness = bestPop["fitness"]
quantileIntersect = bestPop["intersect"]
quantileSimulatedEdges = bestPop["quantileSimulatedEdges"]
bestPop = bestPop["pop"]
monitorStep = 50
L = 10000

## Output results

In [ ]:
indexBestGraph = findmax(map(x -> observedLoss(x, true)[1], bestPop))[2]

In [ ]:
fileGML = open("ARID_best_match_pulldown.gml", "w")
bestStructure = bestPop[findmax(map(x -> observedLoss(x, false)[1], bestPop))[2]]
bestPulldown = structureToPulldown(bestStructure)
#saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, bestPulldown.edges)
close(fileGML)

In [ ]:
fileGML = open("ARID_best_match_structure.gml", "w")
saveLabeledGml(fileGML, pop[indexBestGraph].graph, pop[indexBestGraph].nodes, pop[indexBestGraph].competition)
close(fileGML)

In [ ]:
traceFitness = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceFitness[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileFitness[:,i], mode="lines+markers")
end

layoutFitness = Layout(yaxis_title="<b>Jaccard coefficient distribution</b>", xaxis_title = "<b>Generation</b>")

plot(traceFitness, layoutFitness)

In [ ]:
traceIntersect = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(6)

for i = 1:5
    traceIntersect[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = string("Top ", 25*(i-1), "%"),
        y= quantileIntersect[:,i], mode="lines+markers")
end

traceIntersect[6] = scatter(
    x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = "Edges in observed pull-down graph",
    y= map(x -> length(observedEdges), 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1)), mode="lines")

layoutIntersect = Layout(yaxis_title="<b>Pull-down edges intersection size</b>", xaxis_title = "<b>Generation</b>")

plot(traceIntersect, layoutIntersect)

In [ ]:
traceSimulatedEdges = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceSimulatedEdges[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileSimulatedEdges[:,i], mode="lines+markers")
end

layoutSimulatedEdges = Layout(yaxis_title="<b>Number of simulated pull-down edges</b>", xaxis_title = "<b>Generation</b>")

plot(traceSimulatedEdges, layoutSimulatedEdges)

## Average on whole population

In [ ]:
# Weight by fitness
popWeight = map(x -> observedLoss(x, true)[1], pop)

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i > j)
for i in 1:length(pop)
    graph = pop[i].graph
    competition = pop[i].competition
    for nodeA in 2:nv(graph)
        for nodeB in 1:nv(graph)
            if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                averageComp[(nodeA,nodeB)] += popWeight[i]
            end
        end
    end
end
                
# Remove null values
averageComp = Dict(c => v/sum(popWeight) for (c,v) in averageComp if v > 0)

In [ ]:
# Cumulated weights of the graph having each edge
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i != j)
for i in 1:length(pop)
    for c = edges(pop[i].graph)
        averageEdges[Tuple(c)] += popWeight[i]
    end
end
                
# Remove null values
averageEdges = Dict(c => v/sum(popWeight) for (c,v) in averageEdges if v != 0)

### Export graph with two weighted edge types

In [ ]:
"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and 2 sets of edge labels
'elabs' given in three dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs1::Dict{Tuple{Int64,Int64},Float64}, elabs2::Dict{Tuple{Int64,Int64},Float64})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for (e,v) = elabs1
        s, t = sort(collect(e))
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tweight $v")
        println(io, "\t\tclass 1")
        println(io, "\t]")
    end
    for (e,v) = elabs2
        t, s = sort(collect(e))
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tweight $v")
        println(io, "\t\tclass 2")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

In [ ]:
fileGML = open("ARID_average_structure.gml", "w")
saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, averageEdges, averageComp)
close(fileGML)

### Display infered heatmap

In [ ]:
focusBAFko = [e for e in studyBAFko if e in studyBAFunits]
focusBAFpd = sort(studyBAFpd)

pdSimData = zeros(length(studyBAFpd), length(focusBAFko))

for e in edges(bestPulldown.graph)
    k, v = Tuple(e)
    
    # What type / value for the edge?
    t = bestPulldown.edges[(k,v)] == "inhibits" ? -1 : 1
    
    # Which cell should we fill?
    indexKO = findfirst(focusBAFko, bestPulldown.nodes[k])
    indexPD = findfirst(focusBAFpd, bestPulldown.nodes[v])
                
    pdSimData[indexPD, indexKO] = t
end

pdSimData

In [ ]:
tracePdHeatmap = heatmap(
    x=studyBAFpd,
    y=focusBAFko, # NB: filter genes outside of BAF complex
    z=pdSimData
)

stylePdHeatmap = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [0.5, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutPdHeatmap = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")
plot(tracePdHeatmap, layoutPdHeatmap, style=stylePdHeatmap)

## Compare heatmaps

In [ ]:
binaryObserved = sign.(convert(Array, aridData[:,[i+1 for (i,v) in enumerate(convert.(String, names(aridData[:,2:end]))) if v in studyBAFunits]]))

traceArid = heatmap(
    x=aridData[1],
    y=names(aridData[:,[i+1 for (i,v) in enumerate(convert.(String, names(aridData[:,2:end]))) if v in studyBAFunits]]),
    z=binaryObserved)

coordZero = -minimum(convert(Array, aridData[:,2:end])) /
    (maximum(convert(Array, aridData[:,2:end])) - minimum(convert(Array, aridData[:,2:end])))
styleArid = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [0.5, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutArid = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")

plot(traceArid, layoutArid, style=styleArid)

## Average accross runs

In [ ]:
pop = []
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/randomIni/500x10000/"
for runFile = [i for i in readdir(folder) if contains(i, ".jld")]
    popl = load(folder*runFile)
    append!(pop, popl["pop"])
    println(typeof(popl["pop"]))
    println(length(popl["pop"]))
end

In [ ]:
pop2 = Array{structureGraph,1}()
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/littIni/500x10000/"
for runFile = [i for i in readdir(folder) if contains(i, ".jld")]
    popl = load(folder*runFile)
    append!(pop2, popl["pop"])
    println(typeof(popl["pop"]))
    println(length(popl["pop"]))
end

In [ ]:
quantile(map(x -> observedLoss(x, false)[1], pop))

In [ ]:
quantile(map(x -> observedLoss(x, false)[1], pop2))

In [ ]:
pop = append!(pop, pop2)
quantile(map(x -> observedLoss(x, false)[1], pop))

In [ ]:
pop3 = Array{structureGraph,1}()
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/simiIni/500x10000/"
for runFile = [i for i in readdir(folder) if contains(i, ".jld")]
    popl = load(folder*runFile)
    append!(pop3, popl["pop"])
    println(typeof(popl["pop"]))
    println(length(popl["pop"]))
end

In [ ]:
pop = append!(pop, pop3)
quantile(map(x -> observedLoss(x, false)[1], pop))

In [ ]:
# Weight by fitness
popWeight = map(x -> observedLoss(x, true)[1], pop)

In [ ]:
quantile(map(x -> observedLoss(x, false)[1], pop3))

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i > j)
for i in 1:length(pop)
    graph = pop[i].graph
    competition = pop[i].competition
    for nodeA in 2:nv(graph)
        for nodeB in 1:nv(graph)
            if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                averageComp[(nodeA,nodeB)] += popWeight[i]
            end
        end
    end
end
                
# Remove null values
averageComp = Dict(c => v/sum(popWeight) for (c,v) in averageComp if v > 0)

In [ ]:
# Cumulated weights of the graph having each edge
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i != j)
for i in 1:length(pop)
    for c = edges(pop[i].graph)
        averageEdges[Tuple(c)] += popWeight[i]
    end
end
                
# Remove null values
averageEdges = Dict(c => v/sum(popWeight) for (c,v) in averageEdges if v != 0)

## Tests

In [ ]:
fileGML = open("ARID_average_structure.gml", "w")
saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, averageEdges, averageComp)
close(fileGML)

In [ ]:
pop3 = Array{structureGraph,1}()
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/all005"
allFiles = readdir(folder)
nbFiles = length(allFiles)

for runFile = [i for i in  if contains(i, ".jld")]
    popl = load(folder*runFile)
    append!(pop3, popl["pop"])
    println(typeof(popl["pop"]))
    println(length(popl["pop"]))
end

In [ ]:
pop_01 = Array{structureGraph,1}()
maxFit_01 = 0
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/"
bestPop_01 = Dict{String,Any}()
for runFile = [i for i in readdir(folder) if contains(i, "_01_")]
    println(runFile)
    popl = load(folder*runFile)
    append!(pop_01, popl["pop"])
    maxFitPopl = maximum(map(x -> observedLoss(x, false)[1], popl["pop"]))
    if maxFitPopl > maxFit_01
        bestPop_01 = popl
        maxFit_01 = maxFitPopl
    end
end

In [ ]:
pop_001 = Array{structureGraph,1}()
maxFit_001 = 0
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/"
bestPop_001 = Dict{String,Any}()
for runFile = [i for i in readdir(folder) if contains(i, "_001_")]
    println(runFile)
    popl = load(folder*runFile)
    append!(pop_001, popl["pop"])
    maxFitPopl = maximum(map(x -> observedLoss(x, false)[1], popl["pop"]))
    if maxFitPopl > maxFit_001
        bestPop_001 = popl
        maxFit_001 = maxFitPopl
    end
end

In [ ]:
pop_0005 = Array{structureGraph,1}()
maxFit_0005 = 0
folder = "/Volumes/lvulliard-1/Documents/BAF-structure/"
bestPop_0005 = Dict{String,Any}()
for runFile = [i for i in readdir(folder) if contains(i, "_0005_")]
    println(runFile)
    popl = load(folder*runFile)
    append!(pop_0005, popl["pop"])
    maxFitPopl = maximum(map(x -> observedLoss(x, false)[1], popl["pop"]))
    if maxFitPopl > maxFit_0005
        bestPop_0005 = popl
        maxFit_0005 = maxFitPopl
    end
end

In [ ]:
colnames = ["Units"    
 "ACTB"     
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCA4.4"
 "SMARCA4.6"
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]

edgeTypes = Dict{Float64, Dict{Tuple, String}}()

for alpha = [0.1, 0.05, 0.01, 0.005]    
    aridData = CSV.read("ARID1A-data.csv"; delim='\t', header=colnames, datarow=2)

    foreach(x -> aridData[x] = log2.(aridData[x]), names(aridData[:,2:end]))

    for i in 2:length(aridData)
        for j in 1:length(aridData[i])
            # Some values were stored as factors instead of floats, and could not be compared to ALPHA
            try
                if aridPval[j,i] > alpha
                    aridData[j,i] = 0
                end
            catch e
                if isa(e, MethodError) # In case of type error when comparing the variable to ALPHA 
                    if float(string(aridPval[j,i])) > alpha # Try converting the faulty variable
                        aridData[j,i] = 0
                    end
                end
            end
        end
    end
    
    println(countnz(convert(Array,aridData)))

    # Join SMARCA4.4 and SMARCA4.6
    delete!(aridData, Symbol("SMARCA4.6"))
    rename!(aridData, Symbol("SMARCA4.4") => :SMARCA4)
    
    unitDict = Dict(s => i for (i,s) in enumerate(sort(union(studyBAFko, studyBAFpd))))

    # Store the sign of the log2-fold-change associated with each link
    edgeTypes[alpha] = Dict{Tuple, String}()

    # Parse each column
    for x = names(aridData[:,2:end])
        for y = 1:length(aridData[x])
            if aridData[y,x] < 0
                edgeTypes[alpha][(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "inhibits"
            elseif aridData[y,x] > 0
                edgeTypes[alpha][(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "enhances"
            end
        end
    end
end

In [ ]:
observedEdgesAlpha = Dict(alpha => Dict((convertUnitIndex[u[1]], convertUnitIndex[u[2]]) => v for 
    (u,v) in edgeTypes[alpha] if u[1] in keys(convertUnitIndex) && u[2] in keys(convertUnitIndex)) for alpha in [0.1, 0.05, 0.01, 0.005])
            
"""
Compute loss for a given structure
compared to observation
"""
function observedLoss(sGraph::structureGraph,
    details::Bool = false, alpha::Float64 = 0.05)
    pGraph = structureToPulldown(sGraph)
    
    observedEdges = observedEdgesAlpha[alpha]
    
    intersectEdges = intersect(pGraph.edges, observedEdges)
    unionEdges = union(pGraph.edges, observedEdges)
    
    if details
        # Return array with Jaccard index
        # length of union and length of  
        return([length(intersectEdges) / length(unionEdges), length(intersectEdges), length(pGraph.edges)])
    else
        # Return Jaccard index
        return([length(intersectEdges) / length(unionEdges)])
    end
end

In [ ]:
# Average across alphas
averageAllEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i != j)
averageAllComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i > j)

for alpha = [0.1, 0.05, 0.01, 0.005]
    pop = eval(parse("pop_"* replace(string(alpha), ".", "")))
    popWeight = map(x -> observedLoss(x, false, alpha)[1], pop)
    sumWeight = sum(popWeight)
    println(quantile(popWeight))
    
    # Cumulated weights of the graph having each edge
    averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i != j)
    for i in 1:length(pop)
        for c = edges(pop[i].graph)
            averageEdges[Tuple(c)] += popWeight[i]
            averageAllEdges[Tuple(c)] += popWeight[i]/sumWeight
        end
    end
                    
    averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i > j)
    for i in 1:length(pop)
        graph = pop[i].graph
        competition = pop[i].competition
        for nodeA in 2:nv(graph)
            for nodeB in 1:nv(graph)
                if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                    averageComp[(nodeA,nodeB)] += popWeight[i]
                    averageAllComp[(nodeA, nodeB)] += popWeight[i]/sumWeight
                end
            end
        end
    end
                
    # Remove null values
    averageComp = Dict(c => v/sumWeight for (c,v) in averageComp if v > 0)
    averageEdges = Dict(c => v/sumWeight for (c,v) in averageEdges if v > 0)
                                                                                                
    fileGML = open("ARID_average_structure_"*string(alpha)*".gml", "w")
    #saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, averageEdges, averageComp)
    close(fileGML)
end

In [ ]:
averageAllComp = Dict(c => v/4 for (c,v) in averageAllComp if v > 0)
averageAllEdges = Dict(c => v/4 for (c,v) in averageAllEdges if v > 0)   
                        
fileGML = open("ARID_average_structure.gml", "w")
saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, averageAllEdges, averageAllComp)
close(fileGML)

In [ ]:
for i = 1:10
    @time begin
        i1 = intersect(pgraph1.edges, observedEdges)
        i2 = intersect(pgraph2.edges, observedEdges)
        print(length(i1)+length(i2))
    end
end

In [ ]:
for i = 1:10
    @time begin
        i1 = intersect(pgraph1.edges, observedEdges)
        append!(i1, intersect(pgraph2.edges, observedEdges))
        print(length(i1))
    end
end

In [ ]:
averageMat = Array{Float64}(M,M)
averageMat .= 0

# Alphabetical order of units
alphaOrderUnits = Dict(v => i for (i,v) in enumerate(sort(studyBAFunits)))
aoUnit = function(x::Int64)
    return(alphaOrderUnits[bestPulldown.nodes[x]])
end

# Fill competition under the diagonal of the matrix
# i.e. x < y
for (t,v) = averageAllComp
    x,y = sort(aoUnit.(collect(t)))
    averageMat[x,y] = v
end

# Fill connections above the diagonal of the matrix
# i.e. x > y
for (t,v) = averageAllEdges
    y,x = sort(aoUnit.(collect(t)))
    averageMat[x,y] = -v
end

traceAverage = heatmap(
    x=sort(studyBAFunits),
    y=sort(studyBAFunits),
    z=averageMat)

styleAverage = Style(global_trace=attr(colorscale=[[0, "rgb(0,140,160)"],
            [minimum(averageMat)/(minimum(averageMat)-maximum(averageMat)), "rgb(255,255,255)"], [1, "rgb(210,50,60)"]]))
layoutAverage = Layout(;margin_l = 90, margin_t = 10, margin_b = 80, yaxis_title="", xaxis_tickangle = -45,
    xaxis_title = "<b>Interaction</b>", yaxis_title = "<b>Competition</b>", font_family="arial", font_size=11)

plot(traceAverage, layoutAverage, style=styleAverage)

On cluster
 - BioAlignments                 0.2.0
 - BioSequences                  0.8.3
 - CSV                           0.2.2
 - DataFrames                    0.11.5
 - Distributions                 0.15.0
 - DocOpt                        0.3.0
 - GraphIO                       0.2.0
 - HDF5                          0.8.8
 - HttpParser                    0.3.1
 - LightGraphs                   0.12.0
 - Plotly                        0.1.1
 - StatsBase                     0.20.1

In [ ]:
Pkg.status()

In [ ]:
versioninfo()